In [22]:
!pip3 install implicit --upgrade
!pip3 install surprise --upgrade

In [23]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import implicit
import scipy.sparse as sparse
import surprise

In [24]:
articles = 'articles.xlsx'
customers = 'customers.xlsx'
transaction_val = 'transactions_val.xlsx'
users_test = 'users_test.xlsx'
transactions_train = 'transactions_train.csv'

In [25]:
df_articles = pd.read_excel(articles)
df_customers = pd.read_excel(customers)
df_transaction_val = pd.read_excel(transaction_val)
df_users_test = pd.read_excel(users_test)
df_transaction_train = pd.read_csv(transactions_train)

Recomendar los 20 productos más populares

In [26]:
!pip3 install pyreclab --upgrade

In [27]:
db_frecuency = df_transaction_train.groupby("article_id").count()
db_frecuency.sort_values(by="price", ascending=False, inplace=True)
db_frecuency_count = db_frecuency["price"]

total = db_frecuency_count.sum()
print(f'Sum of transactions: {total}')

db_frecuency = db_frecuency[db_frecuency['price'] >= 17]
db_frecuency_count = db_frecuency["price"]
sample = db_frecuency_count.sum()
print(f'Sample of transactions: {sample}')
print(f'Percentage of the sample: {(sample / total) * 100}%')

final_articles = list(db_frecuency.index)
print(f'Number of articles in sample: {len(list(final_articles))}')

Sum of transactions: 1318501
Sample of transactions: 1059878
Percentage of the sample: 80.38507365561345%
Number of articles in sample: 20532


In [28]:
user_items_test = {}

for row in df_transaction_val.itertuples():
    if row[3] not in user_items_test:
        user_items_test[row[3]] = []

    if row[4] in final_articles:
        user_items_test[row[3]].append(row[4])

In [29]:
from scipy import sparse
import numpy as np

user_items = {}
itemset = set()

# Construir user_items y itemset
for row in df_transaction_train.itertuples():
    if row[3] not in user_items:
        user_items[row[3]] = []
    user_items[row[3]].append(row[4])
    itemset.add(row[4])

itemset = np.sort(list(itemset))

# Inicializar listas para almacenar filas, columnas y valores de la matriz
rows = []
cols = []
vals = []

# Llenar las listas correspondientes con los valores de la matriz
for i, items in enumerate(user_items.values()):
    for item in items:
        rows.append(itemset.searchsorted(item))
        cols.append(i)
        vals.append(1)

# Crear la matriz dispersa CSR
user_item_matrix = sparse.csr_matrix((vals, (rows, cols)), shape=(len(itemset), len(user_items)))

# Transponer la matriz para tener usuarios como filas y artículos como columnas
user_item_matrix = user_item_matrix.T.tocsr()

In [30]:
import random

In [31]:
# obtener recomendacion de 20 items aleatorios de df_transaction_val
aleatorio = random.sample(list(df_transaction_val['article_id']), 20)
aleatorio

[562498002,
 825720001,
 852678002,
 768440002,
 780510001,
 861040001,
 270381003,
 636323026,
 783245001,
 763735003,
 837810001,
 741356003,
 826508003,
 806534002,
 542533001,
 816166001,
 924243001,
 507883014,
 751471001,
 570003075]

In [32]:
# Obtenido de https://gist.github.com/bwhite/3726239

def precision_at_k(r, k):
    assert k >= 1
    r = np.asarray(r)[:k] != 0
    if r.size != k:
        raise ValueError('Relevance score length < k')
    return np.mean(r)

def average_precision(r):
    r = np.asarray(r) != 0
    out = [precision_at_k(r, k + 1) for k in range(r.size) if r[k]]
    if not out:
        return 0.
    return np.mean(out)

def dcg_at_k(r, k):
    r = np.asfarray(r)[:k]
    if r.size:
        return np.sum(np.subtract(np.power(2, r), 1) / np.log2(np.arange(2, r.size + 2)))
    return 0.

def mean_average_precision(rs):
    return np.mean([average_precision(r) for r in rs])

def ndcg_at_k(r, k):
    idcg = dcg_at_k(sorted(r, reverse=True), k)

    if not idcg:
        return 0.
    return dcg_at_k(r, k) / idcg

In [33]:
def evaluate_model_random(n):
    mean_map = 0.
    mean_ndcg = 0.
    for u in user_items_test.keys():
        aleatorio = []
        while len(aleatorio) < n:
            item = random.choice(list(itemset))
            if item not in aleatorio:
                aleatorio.append(item)
        indices = []
        for item in user_items_test[u]:
            indices.append(list(itemset).index(item))
        rel_vector = [np.isin(indices, aleatorio, assume_unique=True).astype(int)]
        mean_map += mean_average_precision(rel_vector)
        mean_ndcg += ndcg_at_k(rel_vector, n)

    mean_map /= len(user_items_test)
    mean_ndcg /= len(user_items_test)

    return mean_map, mean_ndcg

In [34]:
def evaluate_model_popular(n):
    mean_map = 0.
    mean_ndcg = 0.
    rec = []
    count = 0
    most_popular = []
    for item in df_transaction_train.iterrows():
        if count < 10:
            most_popular.append(item[0])
            count += 1
    for u in user_items_test.keys():
        indices = []
        for item in user_items_test[u]:
            indices.append(list(itemset).index(item))
        rel_vector = [np.isin(indices, rec, assume_unique=True).astype(int)]
        mean_map += mean_average_precision(rel_vector)
        mean_ndcg += ndcg_at_k(rel_vector, n)

    mean_map /= len(user_items_test)
    mean_ndcg /= len(user_items_test)

    return mean_map, mean_ndcg

In [ ]:
map20, ndcg20 = evaluate_model_random(20)
map30, ndcg30 = evaluate_model_random(30)

print('map20:',map20)
print('ndcg20:', ndcg20)

print('map30:',map30)
print('ndcg30:', ndcg30)
# n = 10
# map = 0.0
# ndcg = 0.0

# n = 20
# map = 0.05594000845306313
# ndcg = 0.08463293580787053

# n = 30
# map = 0.05594000845306313
# ndcg = 0.08463293580787053



In [ ]:
map10, ndcg10 = evaluate_model_popular(10)
map20, ndcg20 = evaluate_model_popular(20)
map30, ndcg30 = evaluate_model_popular(30)

print('map10:',map10)
print('ndcg10:', ndcg10)

print('map20:',map20)
print('ndcg20:', ndcg20)

print('map30:',map30)
print('ndcg30:', ndcg30)